# Calculate velocity distribution and position destribution of MDCK spheroids

Data  
2D time-series images of flowing spheroids  
Conditions of experimes (four in total):
- Hydrodynamic focusing (=hydro): on/off
- Acoustic focusing (=acoustic): on/off

Use only frames from 0 to 360 (=nine seconds)

## Libraries

In [ ]:
import glob
import os
import re
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from skimage.io import imread, imshow
from skimage.filters import median, gaussian

import trackpy as tp

## Functions

In [ ]:
def make_dataset(acoustic, hydro, measurement, start_frame=0, end_frame=360):
    '''
    Make a list of data to analyze

    acousitc: whether acoustic focsuing is on or off
    hydro: whether hydrodynamic focusing is on or off
    measurement: id of the measurement to analyze
    start_frame: first frame of the data to analyze
    end_frame: last frame of the data to analyze
    '''
    imgs = [os.path.abspath(p) for p in glob.glob(f'{path_data}id553_acoustic-{acoustic}_flow-{hydro}_{measurement}*_*.tif')]
    imgs = sorted(imgs, key = lambda x : re.search(r'_00[0-9][0-9][0-9].', x).group())
    imgs = imgs[start_frame:end_frame]
    imgs = [imread (img) for img in imgs]
    
    return imgs

def clip(img, bounds=(None, None)):
    '''
    Clip image along the first axis so that the image includes only the inside of the microfluidic channel

    img: image to process
    bounds: coordinates on which image is clipped
    '''
    return img[bounds[0]:bounds[1], :]

def filter_img(img):
    '''
    Apply median and Gaussian filter to image
    '''
    filtered_img = median(img)
    filtered_img = gaussian(filtered_img, sigma=1)
    return filtered_img

def process(imgs, bounds=(None, None)):
    '''
    Processs images before analysis

    imgs: images to analyze
    bounds: coordinates on which images are clipped
    '''
    filtered_imgs = [filter_img(clip(np.fliplr(img), bounds=bounds)) for img in imgs]
    return filtered_imgs

def test(acoustic, hydro, measurement, end_frame=20, bounds=(None, None), threshold=20/255):
    '''
    Analyze the small subset of the data and chack the result

    acousitc: whether acoustic focsuing is on or off
    hydro: whether hydrodynamic focusing is on or off
    measurement: id of the measurement to analyze
    end_frame: last frame of the data to analyze
    bounds: coordinates on which image is clipped
    threshold: threshold value to detect flowing objects
    '''
    imgs = make_dataset(acoustic, hydro, measurement, end_frame=end_frame)
    imgs = process(imgs, bounds)
    locations = tp.batch(imgs, diameter=25, threshold=threshold)
    pred = tp.predict.ChannelPredict(110, 'x', minsamples=10)
    track = pred.link_df(locations, search_range=[13, 40], memory=10)
    track = tp.filter_stubs(track, 5)
    
    fig, axes = plt.subplots(2, 1, figsize=(20, 10))
    tp.annotate(locations[locations['frame']==0], imgs[0], ax=axes[0])
    tp.plot_traj(track)

def analysis(acoustic, hydro, measurement, bounds=(None, None), threshold=20/255):
    '''
    Analyze the first 360 frames of the measurement

    acousitc: whether acoustic focsuing is on or off
    hydro: whether hydrodynamic focusing is on or off
    measurement: id of the measurement to analyze
    bounds: coordinates on which image is clipped
    threshold: threshold value to detect flowing objects
    '''
    imgs = make_dataset(acoustic, hydro, measurement)
    imgs = process(imgs, bounds)
    locations = tp.batch(imgs, diameter=25, threshold=threshold)
    pred = tp.predict.ChannelPredict(110, 'x', minsamples=10)
    track = pred.link_df(locations, search_range=[13, 40], memory=10)
    track = tp.filter_stubs(track, 120)
    imsd = tp.motion.imsd(track, mpp=1/1.124, fps=40, max_lagtime=360)
    print('The number of detected tracks is:', len(imsd.columns))
        
    return track, imsd

## Directories

In [ ]:
path_data = '../../data/2D/uniformity/'
path_result = '../../result/'

## Test using the limited number of frames

In [ ]:
img = imread(path_data+'id553_acoustic-off_flow-off_1*_00001.tif')
plt.figure(figsize=(20, 10))
plt.imshow(img[208:208+1124, :])

In [ ]:
wall_up = 208 # coordinate of the upper wall
wall_bottom = wall_up + 1124 # coordinate of the bottom wall

In [ ]:
test('off', 'off', measurement=1, bounds=(wall_up, wall_bottom))

In [ ]:
test('off', 'on', measurement=1, bounds=(wall_up, wall_bottom))

In [ ]:
test('on', 'off', measurement=1, bounds=(wall_up, wall_bottom))

In [ ]:
test('on', 'on', measurement=1, bounds=(wall_up, wall_bottom))

## Analysis

### acoustic: on & hydro: on

In [ ]:
wall_up = 208
wall_bottom = wall_up + 1124

In [ ]:
on_on_track, on_on_imsd = analysis(acoustic='on', hydro='on', measurement=1, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(on_on_track)

In [ ]:
on_on_imsd.to_csv(path_result+'id553_on_on_1_imsd.csv')
with open(path_result+'id553_on_on_1_track.txt', 'wb') as f:
    pickle.dump(on_on_track, f)

### acoustic: on & hydro: off

In [ ]:
wall_up = 208
wall_bottom = wall_up + 1124

In [ ]:
on_off_track, on_off_imsd = analysis(acoustic='on', hydro='off', measurement=1, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(on_off_track)

In [ ]:
on_off_imsd.to_csv(path_result+'id553_on_off_1_imsd.csv')
with open(path_result+'id553_on_off_1_track.txt', 'wb') as f:
    pickle.dump(on_off_track, f)

### acoustic: off & hydro: off

In [ ]:
wall_up = 208
wall_bottom = wall_up + 1124

In [ ]:
off_off_track, off_off_imsd = analysis(acoustic='off', hydro='off', measurement=1, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(off_off_track)

In [ ]:
off_off_imsd.to_csv(path_result+'id553_off_off_1_imsd.csv')
with open(path_result+'id553_off_off_1_track.txt', 'wb') as f:
    pickle.dump(off_off_track, f)

### acoustic: off & hydro: on

In [ ]:
wall_up = 208
wall_bottom = wall_up + 1124

In [ ]:
off_on_track, off_on_imsd = analysis(acoustic='off', hydro='on', measurement=1, bounds=(wall_up, wall_bottom))

In [ ]:
plt.figure(figsize=(20, 5))
plt.ylim(0, 1124)
tp.plot_traj(off_on_track)

In [ ]:
off_on_imsd.to_csv(path_result+'id553_off_on_1_imsd.csv')
with open(path_result+'id553_off_on_1_track.txt', 'wb') as f:
    pickle.dump(off_on_track, f)